In [ ]:
# updating tf version for Kaggle
!pip install -U tensorflow==2.14.0

In [ ]:
# Set the random seed for reproducibility
seed = 420 

# Set environment variables to control TensorFlow behavior
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TensorFlow logging
os.environ['PYTHONHASHSEED'] = str(seed)  # Set the Python hash seed for reproducibility
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'  # Set the matplotlib config directory

# Ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Set the random seed for numpy
import numpy as np
np.random.seed(seed)

# Configure logging
import logging
logging.basicConfig(level=logging.ERROR)  # Set the logging level to ERROR

# Set the random seed for the random module
import random
random.seed(seed)

In [ ]:
# Import the necessary libraries
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

# Set the verbosity level of TensorFlow to suppress unnecessary logging
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Set the random seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Print the TensorFlow version
print(tf.__version__)

In [ ]:
# Import the necessary libraries
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

# The code above imports the required libraries for image processing, visualization, and evaluation metrics.
# cv2 is used for image processing and manipulation.
# matplotlib.pyplot is used for plotting and visualization.
# train_test_split is used to split the dataset into training and validation sets.
# accuracy_score, f1_score, precision_score, recall_score, and confusion_matrix are used for model evaluation.
# seaborn is used for creating a heatmap to visualize the confusion matrix.


In [ ]:
# Load the datase
dataset = np.load('/kaggle/input/an2dl-homework-data/dataset_wo_duplicates.npz', allow_pickle=True) #file located in private repository

# Define a function to load and preprocess the images from the dataset
def load_images():
    images = []
    for item in dataset['data']:
        img = item
        img = tfkl.Resizing(96,96)(img)  # Resize the image to (96, 96)
        if img is not None:
            images.append(img)
    return np.array(images)

In [ ]:
# Load the images from the dataset
X = load_images()

# Get the labels from the dataset
y = dataset['labels']

# One-hot encoding of the labels
_, y = np.unique(y, return_inverse=True)
y = tfk.utils.to_categorical(y, 2)

In [ ]:
# Initialize counters for the number of healthy and unhealthy samples
num_h, num_u = 0, 0

# Get the total number of samples in the dataset
dim_data = len(dataset["data"])

# Iterate over each sample in the dataset
for i in range(dim_data):
  # Check the label of the sample
  if dataset['labels'][i] == "healthy":
    # Increment the counter for healthy samples
    num_h += 1
  else:
    # Increment the counter for unhealthy samples
    num_u += 1


In [ ]:
# Creating weights fo the classes
weight_h = (1/num_h)*(dim_data/2)
weight_u = (1/num_u)*(dim_data/2)
class_weight = {0: weight_h, 1: weight_u}

In [ ]:
# Define a function to plot the training and validation loss, as well as the training accuracy and validation accuracy
def print_histories(metadata):
  # Plot the training and validation loss
  plt.figure(figsize=(15,5))
  for k in list(metadata.keys()):
    plt.plot(metadata[k]['history']['loss'], alpha=.25, color=metadata[k]['color'][0], linestyle='--')
    plt.plot(metadata[k]['history']['val_loss'], label=k, alpha=.9, color=metadata[k]['color'][0])
  plt.legend(loc='upper left')
  plt.title('Categorical Crossentropy')
  plt.grid(alpha=.15)
  
  # Plot the training and validation accuracy
  plt.figure(figsize=(15,5))
  for k in list(metadata.keys()):
    be = metadata[k]['best_epoch']
    bescore = metadata[k]['history']['val_accuracy'][be]
    plt.plot(metadata[k]['history']['accuracy'], alpha=.25, color=metadata[k]['color'][1], linestyle='--')
    plt.plot(metadata[k]['history']['val_accuracy'], label=k, alpha=.9, color=metadata[k]['color'][1])
    plt.plot(be, bescore, marker='*', color=metadata[k]['color'][1], markersize=15)
    plt.text(0.95*be, 1.02*bescore,
         f'best_epoch={bescore}', fontsize=12, color=metadata[k]['color'][1])
  plt.legend(loc='upper left')
  plt.title('Accuracy')
  plt.grid(alpha=.15)

  plt.show()

In [ ]:
# Split the dataset into training+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=.2, stratify=np.argmax(y,axis=1))

# Further split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=len(X_test), stratify=np.argmax(y_train_val,axis=1))

# The sets are stratified per class hence they keep data distribution, split: 60%/20%/20%

# Delete unnecessary variables to free up memory
del dataset
del X

# Print the shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
# Define the input shape of the training data
input_shape = X_train.shape[1:]

# Define the output shape of the training data
output_shape = y_train.shape[1]

# Define the batch size for training
batch_size = 32

# Define the number of epochs for training
epochs = 2000

# Create an empty dictionary to store metadata
metadata = {}

# Print the input shape, output shape, batch size, and number of epochs
print(f"Input Shape: {input_shape}, Output Shape: {output_shape}, Batch Size: {batch_size}, Epochs: {epochs}")

In [ ]:
# Define the callbacks for early stopping and learning rate reduction
callbacks = [
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, mode='max'),  # Stop training early if validation accuracy does not improve for 20 epochs
    tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, min_delta=1e-5)  # Reduce learning rate if validation loss does not improve for 15 epochs
]

In [ ]:
# Define the ConvNeXtBase model
# - input_shape: The shape of the input images (96x96x3)
# - include_top: Whether to include the fully-connected layer at the top of the network
# - weights: The weight initialization to use for the model
# - pooling: The type of pooling to use after the convolutional layers
conv = tfk.applications.ConvNeXtBase(
    input_shape=input_shape,
    include_top=False,
    weights="imagenet",
    pooling='avg',
)

In [ ]:
# Define the dropout rate
dropout_rate = 0.3

# Use the supernet as feature extractor, i.e. freeze all its weights
conv.trainable = False

# Create an input layer
inputs = tfk.Input(shape=input_shape)

# Define a preprocessing pipeline for data augmentation
preprocessing = tf.keras.Sequential([
   tfkl.RandomRotation(1, seed=seed),
   tfkl.RandomContrast(0.8, seed=seed),
   tfkl.RandomBrightness(0.8, seed=seed),
   tfkl.RandomFlip(seed=seed)
], name='preprocessing')

# Apply the preprocessing pipeline to the input data
preprocess = preprocessing(inputs)

# Connect the preprocessed input to the convnet base
x = conv(preprocess)

# Add a fully connected layer with 512 units and ReLU activation
x = tfkl.Dense(units=512, activation='relu')(x)

# Apply dropout regularization to the previous layer
dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

# Add another fully connected layer with 256 units and ReLU activation
x = tfkl.Dense(units=256, activation='relu')(dropout)

# Apply dropout regularization to the previous layer
dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

# Add a final dense layer with 2 units and softmax activation for classification
outputs = tfkl.Dense(2, activation='softmax')(dropout)

# Create a model connecting the input and output layers
conv_model = tfk.Model(inputs=inputs, outputs=outputs, name='convnext_base')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
conv_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

# Display the model summary
conv_model.summary()

# Print the names of each layer in the convnet base
for i, layer in enumerate(conv_model.get_layer('convnext_base').layers):
   print(i, layer.name)

In [ ]:
# Train the model
conv_history = conv_model.fit(
    x = X_train,  # Input training data
    y = y_train,  # Target training data
    batch_size = batch_size,  # Number of samples per gradient update
    epochs = epochs,  # Number of times to iterate over the entire training dataset
    validation_data = (X_val, y_val),  # Data on which to evaluate the loss and any model metrics at the end of each epoch
    callbacks = callbacks,  # List of callbacks to apply during training
    class_weight= class_weight  # Dictionary mapping class indices to a weight for the class
).history  # Training history containing the loss and metrics values at each epoch


In [ ]:
# Add the ConvNeXt Base model, training history, color, and best epoch to the metadata dictionary
metadata['ConvNeXt Base'] = {
    'model': conv_model,  # The ConvNeXt Base model
    'history': conv_history,  # The training history of the model
    'color': ('#007FFF', '#007FFF'),  # The color used for plotting the model's loss and accuracy
    'best_epoch': np.argmax(conv_history['val_accuracy'])  # The epoch with the highest validation accuracy
}

In [ ]:
# Predict the labels for the test data using the trained ConvNeXt Base model
preds = conv_model.predict(X_test, verbose=0)

# Print the shape of the predictions array
print("Predictions Shape:", preds.shape)

In [ ]:
# Calculate the confusion matrix using the true labels and predicted labels
confmat = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1))

# Calculate the accuracy, precision, recall, and F1 score using the true labels and predicted labels
accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')

# Print the accuracy, precision, recall, and F1 score
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

# Create a heatmap of the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(confmat.T, annot=True, fmt="d", xticklabels=list(('Healthy','Unhealthy')), yticklabels=list(('Healthy','Unhealthy')), cmap='Greens')
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
# Print the training and validation loss, as well as the training and validation accuracy
print_histories(metadata)

Fine Tuning the model

In [ ]:
# Set all layers as trainable
conv_model.get_layer('convnext_base').trainable = True

In [ ]:
# Freeze first N layers
N = 189 #chosen tho freeze the first 17 blocks
for i, layer in enumerate(conv_model.get_layer('convnext_base').layers[:N]):
  layer.trainable=False

In [ ]:
# Compile the model
conv_model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

In [ ]:
# Fine-tune the model
conv_history = conv_model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, y_val),
    callbacks = callbacks,
    class_weight=class_weight
).history

In [ ]:
metadata['ConvNeXt Base'] = {
    'model': conv_model,
    'history': conv_history,
    'color': ('#007FFF', '#007FFF'),
    'best_epoch': np.argmax(conv_history['val_accuracy'])
}

In [ ]:
preds = conv_model.predict(X_test, verbose=0)

print("Predictions Shape:", preds.shape)

In [ ]:
confmat = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1))

accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')

print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

plt.figure(figsize=(10, 8))
sns.heatmap(confmat.T,annot=True, fmt="d", xticklabels=list(('Healthy','Unhealthy')), yticklabels=list(('Healthy','Unhealthy')), cmap='Greens')
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
print_histories(metadata)

In [ ]:
# Save the model
conv_model.save('ConvNeXtBase')

The approach followed in this notebook is identical among all the different models used for TL and FT, changin accordingly the imported model through tfk.applications.desired_model() in cell 13 (eventually changing variables/layers/models names too according to the network used) + changing the numer of frozen layers in cell 21 as pleased.